In [2]:
# Check if CUDA recognized

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU count: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
else:
    print("❌ CUDA not detected by PyTorch")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA version: 12.1
GPU count: 1
GPU name: NVIDIA GeForce RTX 2060
GPU memory: 6.0 GB


In [3]:
# Broad model family search

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Mixed precision imports (from your reference code)
try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

# GPU Configuration (matching your reference code)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.backends.cudnn.benchmark = True
    
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
        print("Mixed Precision: Disabled")
else:
    use_amp = False
    print("WARNING: GPU not available")

# Data paths
color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            # Cap ages over 5.5 to 5.5
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            # Skip files with non-numeric age (e.g., "xpx")
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    # Process color images (convert to grayscale)
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            # Convert to 3-channel for pretrained models
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    # Process grayscale images
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            assert img_resized.shape == img_size, f"Image {filename} not resized correctly: {img_resized.shape}"
                            # Convert to 3-channel for pretrained models
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    # Verify final dimensions
    assert images.shape[1:3] == img_size, f"Final image dimensions incorrect: {images.shape}"
    print(f"Images loaded with shape: {images.shape}")
    print(f"Classes: {np.unique(ages)} (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

class DeerDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].clone()
        label = self.y[idx].clone()
        
        # Normalize to [0,1]
        if image.max() > 1.0:
            image = image / 255.0
        
        # Convert to CHW format
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        # Normalize with ImageNet stats
        image = (image - self.mean) / self.std
        
        return image, label

def create_model(model_name, num_classes=5):
    """Create model using timm (matching your reference code)"""
    if model_name == 'ResNet50':
        model = timm.create_model('resnet50', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB0':
        model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=num_classes)
    elif model_name == 'VGG16':
        model = timm.create_model('vgg16', pretrained=True, num_classes=num_classes)
    elif model_name == 'MobileNetV2':
        model = timm.create_model('mobilenetv2_100', pretrained=True, num_classes=num_classes)
    elif model_name == 'InceptionV3':
        model = timm.create_model('inception_v3', pretrained=True, num_classes=num_classes)
    elif model_name == 'DenseNet121':
        model = timm.create_model('densenet121', pretrained=True, num_classes=num_classes)
    elif model_name == 'ResNet101':
        model = timm.create_model('resnet101', pretrained=True, num_classes=num_classes)
    elif model_name == 'ResNet152':
        model = timm.create_model('resnet152', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB1':
        model = timm.create_model('efficientnet_b1', pretrained=True, num_classes=num_classes)
    elif model_name == 'EfficientNetB2':
        model = timm.create_model('efficientnet_b2', pretrained=True, num_classes=num_classes)
    else:
        raise ValueError(f"Unknown model: {model_name}")
    
    return model.to(device)

def train_model(model, train_loader, test_loader, model_name, epochs=50):
    """Train model with your proven GPU configuration"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5, min_lr=1e-6)
    
    best_acc = 0.0
    patience = 10
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} on {device}")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_correct = 0
        train_total = 0
        train_loss_total = 0.0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            train_loss_total += loss.item()
            
            # Memory management (from your reference code)
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        
        # Validation
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                else:
                    outputs = model(images)
                
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_acc = 100 * test_correct / test_total
        scheduler.step(test_acc)
        
        # Early stopping
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 10 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:2d}: Train {train_acc:.1f}%, Test {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        # Memory cleanup
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

# Create timestamped output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_models_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Models will be saved to: {output_dir}")

# Load data
print("Loading images...")
X, y = load_images(color_path, grayscale_path)
print(f"Loaded {len(X)} images, age range: {y.min():.1f}-{y.max():.1f}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets
train_dataset = DeerDataset(X_train, y_train)
test_dataset = DeerDataset(X_test, y_test)

# Create dataloaders (using your batch size from reference code)
batch_size = 16  # From your reference code for RTX 2060
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Model families to test
model_families = [
    'ResNet50', 'EfficientNetB0', 'VGG16', 
    'MobileNetV2', 'InceptionV3', 'DenseNet121'
]

results = []
best_accuracy = 0
best_family = None

print(f"\nTesting {len(model_families)} model families...")

for model_name in model_families:
    print(f"\nTesting {model_name}...")
    
    try:
        model = create_model(model_name)
        trained_model, accuracy = train_model(
            model, train_loader, test_loader, model_name
        )
        
        # Save model with accuracy in filename
        acc_str = f"{accuracy:.3f}".replace('.', 'p')
        model_filename = f"{model_name}_{acc_str}.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': model_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        results.append({
            'model': model_name,
            'accuracy': accuracy,
            'filename': model_filename,
            'full_path': model_path
        })
        
        print(f"{model_name}: Accuracy={accuracy:.3f}, Saved: {model_filename}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_family = model_name
        
        # Cleanup (from your reference code)
        del model, trained_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {model_name}: {e}")
        continue

# Find best performing family
print(f"\nBest family: {best_family} (accuracy: {best_accuracy:.3f})")

# Test variations within best family
if best_family:
    print(f"\nTesting variations of {best_family}...")
    
    if best_family == 'ResNet50':
        variations = ['ResNet101', 'ResNet152']
    elif best_family == 'EfficientNetB0':
        variations = ['EfficientNetB1', 'EfficientNetB2']
    else:
        variations = []
    
    for var_name in variations:
        print(f"\nTesting {var_name}...")
        
        try:
            model = create_model(var_name)
            trained_model, accuracy = train_model(
                model, train_loader, test_loader, var_name
            )
            
            # Save model
            acc_str = f"{accuracy:.3f}".replace('.', 'p')
            model_filename = f"{var_name}_{acc_str}.pth"
            model_path = os.path.join(output_dir, model_filename)
            
            torch.save({
                'model_state_dict': trained_model.state_dict(),
                'model_name': var_name,
                'accuracy': accuracy,
                'num_classes': 5
            }, model_path)
            
            results.append({
                'model': var_name,
                'accuracy': accuracy,
                'filename': model_filename,
                'full_path': model_path
            })
            
            print(f"{var_name}: Accuracy={accuracy:.3f}, Saved: {model_filename}")
            
            # Cleanup
            del model, trained_model
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"Error with {var_name}: {e}")
            continue

# Final results
print(f"\n{'='*50}")
print("FINAL RESULTS")
print(f"{'='*50}")

results.sort(key=lambda x: x['accuracy'], reverse=True)

for i, result in enumerate(results, 1):
    print(f"{i:2d}. {result['model']:15s} - Accuracy: {result['accuracy']:.3f}")

if results:
    best_result = results[0]
    print(f"\nBest model: {best_result['model']} with {best_result['accuracy']:.3f} accuracy")
    print(f"Saved as: {best_result['filename']}")

print(f"\nTotal models tested: {len(results)}")
print(f"All models saved in folder: {output_dir}")
print("All models saved with accuracy in filename.")

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 6.0 GB
Mixed Precision: Enabled
Models will be saved to: deer_age_models_20250731_072915
Loading images...
Images loaded with shape: (466, 224, 224, 3)
Classes: [0 1 2 3 4] (0=1.5yr, 1=2.5yr, 2=3.5yr, 3=4.5yr, 4=5.5yr)
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Loaded 466 images, age range: 0.0-4.0
Train: 372, Test: 94

Testing 6 model families...

Testing ResNet50...
Training ResNet50 on cuda
  Epoch  0: Train 22.6%, Test 25.5%
  Epoch 10: Train 100.0%, Test 47.9%
  Epoch 19: Train 100.0%, Test 50.0%
  Early stopping at epoch 19
ResNet50: Accuracy=51.064, Saved: ResNet50_51p064.pth

Testing EfficientNetB0...
Training EfficientNetB0 on cuda
  Epoch  0: Train 29.6%, Test 35.1%
  Epoch 10: Train 100.0%, Test 50.0%
  Epoch 18: Train 100.0%, Test 55.3%
  Early stopping at epoch 18
EfficientNetB0: Accuracy=55.319, Saved: EfficientNetB0_55p319.pth

Testing VGG

In [5]:
# Deeper family search.

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

# GPU Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    torch.backends.cudnn.benchmark = True
    
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

# Data paths
color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def augment_image(image):
    """Enhanced augmentation to reduce overfitting"""
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.7:
        angle = random.uniform(-15, 15)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.8:
        alpha = random.uniform(0.7, 1.3)
        beta = random.randint(-25, 25)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    if random.random() < 0.3:
        noise = np.random.normal(0, 8, image.shape).astype(np.int16)
        image_int16 = image.astype(np.int16)
        noisy_image = np.clip(image_int16 + noise, 0, 255)
        image = noisy_image.astype(np.uint8)
    
    return image

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Images loaded with shape: {images.shape}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

class AugmentedDeerDataset(Dataset):
    def __init__(self, X, y, augment=False):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        # Apply augmentation during training
        if self.augment:
            image = augment_image(image)
        
        # Convert to tensor and normalize
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        return image, label

def create_model_with_regularization(model_name, num_classes=5, dropout_rate=0.5):
    """Create model with better regularization"""
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=dropout_rate)
    
    # Freeze more layers to reduce overfitting
    if 'resnet' in model_name:
        for name, param in model.named_parameters():
            if not ('layer4' in name or 'fc' in name):
                param.requires_grad = False
    elif 'efficientnet' in model_name:
        for name, param in model.named_parameters():
            if not ('blocks.6' in name or 'blocks.7' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'densenet' in model_name:
        for name, param in model.named_parameters():
            if not ('denseblock4' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'mobilenet' in model_name:
        for name, param in model.named_parameters():
            if not ('features.18' in name or 'features.19' in name or 'classifier' in name):
                param.requires_grad = False
    
    return model.to(device)

def train_model_improved(model, train_loader, test_loader, model_name, epochs=60):
    """Improved training with better regularization"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.15)
    
    # Lower learning rate and higher weight decay
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    best_acc = 0.0
    patience = 15
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} with improved regularization")
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            
            if use_amp:
                with autocast():
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = model(images)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            
            _, predicted = torch.max(outputs, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Validation
        model.eval()
        test_correct = 0
        test_total = 0
        
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                else:
                    outputs = model(images)
                
                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_acc = 100 * test_correct / test_total
        
        # Early stopping
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 10 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:2d}: Train {train_acc:.1f}%, Test {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

# Create timestamped output folder
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_deep_survey_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Models will be saved to: {output_dir}")

# Load data
print("Loading images...")
X, y = load_images(color_path, grayscale_path)
print(f"Loaded {len(X)} images")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets with augmentation
train_dataset = AugmentedDeerDataset(X_train, y_train, augment=True)
test_dataset = AugmentedDeerDataset(X_test, y_test, augment=False)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

# Deep exploration of top 3 families
model_configs = [
    # DenseNet family (won previous round)
    ('densenet121', 'DenseNet121'),
    ('densenet169', 'DenseNet169'),
    ('densenet201', 'DenseNet201'),
    
    # EfficientNet family (2nd place)
    ('efficientnet_b0', 'EfficientNetB0'),
    ('efficientnet_b1', 'EfficientNetB1'),
    ('efficientnet_b2', 'EfficientNetB2'),
    ('efficientnet_b3', 'EfficientNetB3'),
    
    # MobileNet family (3rd place)
    ('mobilenetv2_100', 'MobileNetV2'),
    ('mobilenetv3_small_100', 'MobileNetV3Small'),
    ('mobilenetv3_large_100', 'MobileNetV3Large'),
    
    # Additional high-performers to test
    ('resnet50', 'ResNet50_Regularized'),
    ('resnext50_32x4d', 'ResNeXt50'),
]

results = []
print(f"\nDeep survey: Testing {len(model_configs)} models with improved regularization...")

for model_timm_name, display_name in model_configs:
    print(f"\nTesting {display_name}...")
    
    try:
        model = create_model_with_regularization(model_timm_name, dropout_rate=0.5)
        trained_model, accuracy = train_model_improved(
            model, train_loader, test_loader, display_name
        )
        
        # Save model
        acc_str = f"{accuracy:.3f}".replace('.', 'p')
        model_filename = f"{display_name}_{acc_str}.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        results.append({
            'model': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'filename': model_filename,
            'full_path': model_path
        })
        
        print(f"{display_name}: {accuracy:.3f}% - Saved: {model_filename}")
        
        # Cleanup
        del model, trained_model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        gc.collect()
        
    except Exception as e:
        print(f"Error with {display_name}: {e}")
        continue

# Final results
print(f"\n{'='*60}")
print("DEEP SURVEY RESULTS - TOP 3 FAMILIES + EXTRAS")
print(f"{'='*60}")

results.sort(key=lambda x: x['accuracy'], reverse=True)

print("DENSENET FAMILY:")
for result in results:
    if 'DenseNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nEFFICIENTNET FAMILY:")
for result in results:
    if 'EfficientNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nMOBILENET FAMILY:")
for result in results:
    if 'MobileNet' in result['model']:
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print("\nOTHER MODELS:")
for result in results:
    if not any(family in result['model'] for family in ['DenseNet', 'EfficientNet', 'MobileNet']):
        print(f"  {result['model']:20s} - {result['accuracy']:.3f}%")

print(f"\n{'='*60}")
print("OVERALL RANKING:")
for i, result in enumerate(results, 1):
    print(f"{i:2d}. {result['model']:20s} - {result['accuracy']:.3f}%")

if results:
    best_result = results[0]
    print(f"\nBEST MODEL: {best_result['model']} - {best_result['accuracy']:.3f}%")
    print(f"Saved as: {best_result['filename']}")

print(f"\nTotal models tested: {len(results)}")
print(f"All models saved in: {output_dir}")
print("Note: Improved regularization should reduce train/test accuracy gap")

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
GPU Memory: 6.0 GB
Mixed Precision: Enabled
Models will be saved to: deer_age_deep_survey_20250731_075454
Loading images...
Images loaded with shape: (466, 224, 224, 3)
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Loaded 466 images
Train: 372, Test: 94

Deep survey: Testing 12 models with improved regularization...

Testing DenseNet121...
Training DenseNet121 with improved regularization
  Epoch  0: Train 25.5%, Test 30.9%
  Epoch 10: Train 85.8%, Test 56.4%
  Epoch 20: Train 91.1%, Test 46.8%
  Epoch 25: Train 95.7%, Test 47.9%
  Early stopping at epoch 25
DenseNet121: 56.383% - Saved: DenseNet121_56p383.pth

Testing DenseNet169...
Training DenseNet169 with improved regularization
  Epoch  0: Train 27.7%, Test 34.0%
  Epoch 10: Train 90.3%, Test 52.1%
  Epoch 20: Train 95.7%, Test 60.6%
  Epoch 30: Train 98.9%, Test 60.6%
  Epoch 40: Train 99.7%, Test 63.8%
  Epoch 4

In [7]:
# Second attempt at model families 

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import timm
import numpy as np
import cv2
import os
import gc
import random
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

try:
    from torch.cuda.amp import autocast, GradScaler
    MIXED_PRECISION_AVAILABLE = True
except ImportError:
    MIXED_PRECISION_AVAILABLE = False
    class autocast:
        def __enter__(self):
            return self
        def __exit__(self, *args):
            pass

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.backends.cudnn.benchmark = True
    if MIXED_PRECISION_AVAILABLE:
        scaler = GradScaler()
        use_amp = True
        print("Mixed Precision: Enabled")
    else:
        use_amp = False
else:
    use_amp = False

color_path = r"G:\Dropbox\AI Projects\buck\images\squared\color"
grayscale_path = r"G:\Dropbox\AI Projects\buck\images\squared\grayscale"

def parse_filename(filename):
    parts = filename.split('_')
    if len(parts) >= 4:
        age_str = parts[3]
        try:
            age = float(age_str.replace('p', '.'))
            if age > 5.5:
                age = 5.5
            return age
        except ValueError:
            return None
    return None

def age_to_class(age):
    age_mapping = {1.5: 0, 2.5: 1, 3.5: 2, 4.5: 3, 5.5: 4}
    return age_mapping.get(age, None)

def mixup_data(x, y, alpha=0.4):
    """Mixup augmentation to create synthetic training examples"""
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(device)
    
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    """Mixup loss function"""
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def load_images(color_path, grayscale_path, img_size=(224, 224)):
    images = []
    ages = []
    
    if os.path.exists(color_path):
        for filename in os.listdir(color_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(color_path, filename)
                        img = cv2.imread(img_path)
                        if img is not None:
                            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                            img_resized = cv2.resize(img_gray, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    if os.path.exists(grayscale_path):
        for filename in os.listdir(grayscale_path):
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                age = parse_filename(filename)
                if age is not None:
                    class_idx = age_to_class(age)
                    if class_idx is not None:
                        img_path = os.path.join(grayscale_path, filename)
                        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                        if img is not None:
                            img_resized = cv2.resize(img, img_size)
                            img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_GRAY2RGB)
                            images.append(img_rgb)
                            ages.append(class_idx)
    
    images = np.array(images)
    ages = np.array(ages)
    
    print(f"Total images: {len(images)}")
    print(f"Class distribution: {Counter(ages)}")
    
    return images, ages

def conservative_augment(image):
    """Very light augmentation to preserve deer features"""
    if random.random() < 0.5:
        image = cv2.flip(image, 1)
    
    if random.random() < 0.3:
        angle = random.uniform(-8, 8)
        h, w = image.shape[:2]
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        image = cv2.warpAffine(image, M, (w, h))
    
    if random.random() < 0.4:
        alpha = random.uniform(0.9, 1.1)
        beta = random.randint(-10, 10)
        image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    
    return image

class MultiScaleDataset(Dataset):
    def __init__(self, X, y, augment=False, scale_size=224):
        self.X = X
        self.y = torch.LongTensor(y)
        self.augment = augment
        self.scale_size = scale_size
        self.mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        self.std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx].copy()
        label = self.y[idx].clone()
        
        # Multi-scale training
        if self.augment:
            scale_factor = random.choice([0.8, 0.9, 1.0, 1.1, 1.2])
            new_size = int(self.scale_size * scale_factor)
            image = cv2.resize(image, (new_size, new_size))
            image = cv2.resize(image, (self.scale_size, self.scale_size))
            
            image = conservative_augment(image)
        
        image = torch.FloatTensor(image)
        if image.max() > 1.0:
            image = image / 255.0
        
        if len(image.shape) == 3 and image.shape[-1] == 3:
            image = image.permute(2, 0, 1)
        
        image = (image - self.mean) / self.std
        
        return image, label

def create_conservative_model(model_name, num_classes=5):
    """Back to simpler model creation that worked"""
    model = timm.create_model(model_name, pretrained=True, num_classes=num_classes, drop_rate=0.4)
    
    # Conservative freezing (like the 63.8% model)
    if 'densenet' in model_name:
        for name, param in model.named_parameters():
            if not ('denseblock4' in name or 'classifier' in name):
                param.requires_grad = False
    elif 'resnext' in model_name or 'resnet' in model_name:
        for name, param in model.named_parameters():
            if not ('layer4' in name or 'fc' in name):
                param.requires_grad = False
    
    return model.to(device)

def train_with_mixup_and_multiscale(model, train_loader, test_loader, model_name, epochs=120):
    """Training with mixup + multi-scale + very conservative approach"""
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    # Conservative optimizer (back to what worked)
    optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.05)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-6)
    
    best_acc = 0.0
    patience = 30
    patience_counter = 0
    best_state = None
    
    print(f"Training {model_name} with Mixup + Multi-scale")
    
    for epoch in range(epochs):
        # Training with mixup
        model.train()
        train_correct = 0
        train_total = 0
        
        for batch_idx, (images, labels) in enumerate(train_loader):
            images, labels = images.to(device), labels.to(device)
            
            # Apply mixup
            if random.random() < 0.5:  # 50% chance of mixup
                mixed_images, y_a, y_b, lam = mixup_data(images, labels, alpha=0.4)
                optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = model(mixed_images)
                        loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(mixed_images)
                    loss = mixup_criterion(criterion, outputs, y_a, y_b, lam)
                    loss.backward()
                    optimizer.step()
                
                # For accuracy calculation, use original labels
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == y_a).sum().item()
            else:
                # Normal training
                optimizer.zero_grad()
                
                if use_amp:
                    with autocast():
                        outputs = model(images)
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scaler.update()
                else:
                    outputs = model(images)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                
                _, predicted = torch.max(outputs, 1)
                train_total += labels.size(0)
                train_correct += (predicted == labels).sum().item()
            
            if batch_idx % 10 == 0 and torch.cuda.is_available():
                torch.cuda.empty_cache()
        
        train_acc = 100 * train_correct / train_total
        scheduler.step()
        
        # Simple TTA evaluation (not too heavy)
        test_acc = evaluate_with_simple_tta(model, test_loader)
        
        if test_acc > best_acc:
            best_acc = test_acc
            patience_counter = 0
            best_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if epoch % 20 == 0 or patience_counter >= patience:
            print(f"  Epoch {epoch:3d}: Train {train_acc:.1f}%, Test+TTA {test_acc:.1f}%")
        
        if patience_counter >= patience:
            print(f"  Early stopping at epoch {epoch}")
            break
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    if best_state is not None:
        model.load_state_dict(best_state)
    
    return model, best_acc

def evaluate_with_simple_tta(model, test_loader):
    """Simple TTA - just 3 versions"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Original prediction
            if use_amp:
                with autocast():
                    outputs1 = model(images)
            else:
                outputs1 = model(images)
            
            # Horizontal flip
            flipped = torch.flip(images, [3])
            if use_amp:
                with autocast():
                    outputs2 = model(flipped)
            else:
                outputs2 = model(flipped)
            
            # Slight zoom
            zoomed = F.interpolate(images, scale_factor=0.95, mode='bilinear', align_corners=False)
            zoomed = F.interpolate(zoomed, size=(224, 224), mode='bilinear', align_corners=False)
            if use_amp:
                with autocast():
                    outputs3 = model(zoomed)
            else:
                outputs3 = model(zoomed)
            
            # Average predictions
            avg_outputs = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1) + F.softmax(outputs3, dim=1)) / 3
            _, predicted = torch.max(avg_outputs, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

def ensemble_predict(models, test_loader):
    """Simple ensemble of multiple models"""
    all_models_eval = [model.eval() for model in models]
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            ensemble_output = torch.zeros(images.size(0), 5).to(device)
            
            for model in models:
                # Simple TTA for each model
                if use_amp:
                    with autocast():
                        outputs1 = model(images)
                        outputs2 = model(torch.flip(images, [3]))
                else:
                    outputs1 = model(images)
                    outputs2 = model(torch.flip(images, [3]))
                
                avg_model_output = (F.softmax(outputs1, dim=1) + F.softmax(outputs2, dim=1)) / 2
                ensemble_output += avg_model_output
            
            # Final ensemble prediction
            _, predicted = torch.max(ensemble_output, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    return 100 * correct / total

# Main execution
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = f"deer_age_ensemble_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Ensemble models saved to: {output_dir}")

print("Loading images...")
X, y = load_images(color_path, grayscale_path)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Train: {len(X_train)}, Test: {len(X_test)}")

# Create datasets
train_dataset = MultiScaleDataset(X_train, y_train, augment=True)
test_dataset = MultiScaleDataset(X_test, y_test, augment=False)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=0)

print("\n" + "="*60)
print("SMALL DATA STRATEGY: MIXUP + MULTI-SCALE + ENSEMBLE")
print("="*60)
print("Approach: Conservative training + Mixup synthetic data")

# Train multiple models for ensemble
model_configs = [
    ('densenet169', 'DenseNet169'),
    ('resnext50_32x4d', 'ResNeXt50'),
    ('densenet201', 'DenseNet201'),
]

trained_models = []
individual_scores = []

for model_timm_name, display_name in model_configs:
    print(f"\n{'='*40}")
    print(f"Training {display_name}")
    print(f"{'='*40}")
    
    try:
        model = create_conservative_model(model_timm_name)
        trained_model, accuracy = train_with_mixup_and_multiscale(
            model, train_loader, test_loader, display_name
        )
        
        # Save individual model
        acc_str = f"{accuracy:.1f}".replace('.', 'p')
        model_filename = f"{display_name}_{acc_str}pct.pth"
        model_path = os.path.join(output_dir, model_filename)
        
        torch.save({
            'model_state_dict': trained_model.state_dict(),
            'model_name': display_name,
            'timm_name': model_timm_name,
            'accuracy': accuracy,
            'num_classes': 5
        }, model_path)
        
        trained_models.append(trained_model)
        individual_scores.append(accuracy)
        
        print(f"{display_name}: {accuracy:.1f}% - Saved")
        
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        
    except Exception as e:
        print(f"Error with {display_name}: {e}")
        continue

# Ensemble evaluation
if len(trained_models) > 1:
    print(f"\n{'='*40}")
    print("ENSEMBLE EVALUATION")
    print(f"{'='*40}")
    
    ensemble_accuracy = ensemble_predict(trained_models, test_loader)
    
    print("INDIVIDUAL MODEL RESULTS:")
    for i, (score, config) in enumerate(zip(individual_scores, model_configs)):
        print(f"  {config[1]}: {score:.1f}%")
    
    print(f"\nENSEMBLE RESULT: {ensemble_accuracy:.1f}%")
    
    if ensemble_accuracy >= 75.0:
        print("SUCCESS: 75% target achieved!")
    else:
        gap = 75.0 - ensemble_accuracy
        print(f"Gap to 75%: {gap:.1f}%")
        
        if ensemble_accuracy > max(individual_scores):
            improvement = ensemble_accuracy - max(individual_scores)
            print(f"Ensemble improvement: +{improvement:.1f}%")

print(f"\nAll models saved in: {output_dir}")
print("="*60)

Using device: cuda
GPU: NVIDIA GeForce RTX 2060
Mixed Precision: Enabled
Ensemble models saved to: deer_age_ensemble_20250731_220115
Loading images...
Total images: 466
Class distribution: Counter({np.int64(4): 118, np.int64(2): 111, np.int64(3): 89, np.int64(1): 82, np.int64(0): 66})
Train: 372, Test: 94

SMALL DATA STRATEGY: MIXUP + MULTI-SCALE + ENSEMBLE
Approach: Conservative training + Mixup synthetic data

Training DenseNet169
Training DenseNet169 with Mixup + Multi-scale
  Epoch   0: Train 29.8%, Test+TTA 37.2%
  Epoch  20: Train 64.2%, Test+TTA 54.3%
  Epoch  40: Train 80.1%, Test+TTA 56.4%
  Epoch  60: Train 78.5%, Test+TTA 62.8%
  Epoch  64: Train 81.5%, Test+TTA 57.4%
  Early stopping at epoch 64
DenseNet169: 64.9% - Saved

Training ResNeXt50
Training ResNeXt50 with Mixup + Multi-scale
  Epoch   0: Train 23.1%, Test+TTA 24.5%
  Epoch  20: Train 84.9%, Test+TTA 53.2%
  Epoch  40: Train 84.7%, Test+TTA 52.1%
  Epoch  58: Train 84.4%, Test+TTA 57.4%
  Early stopping at epoch 58

In [ ]:
# Third attempt at model families